#**Lab 06 Softmax Zoo_classifier-eager**

* Softmax를 사용하여 Zoo 데이터를 활용하여 분류를 진행


##**기본 Library 선언 및 Tensorflow 버전 확인**

In [1]:
import tensorflow as tf
import numpy as np

print(tf.__version__)

tf.random.set_seed(777) # for reproducibility

2.4.0


In [3]:
from google.colab import files
uploaded = files.upload()

Saving data-04-zoo.csv to data-04-zoo.csv


In [4]:
xy = np.loadtxt('data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, -1]

nb_classes = 7 # 0~6

# Make Y data as onehot shape
Y_one_hot = tf.one_hot(y_data.astype(np.int32), nb_classes)

print(x_data.shape, Y_one_hot.shape)

(101, 16) (101, 7)


In [8]:
#Weight and bias setting
W = tf.Variable(tf.random.normal((16, nb_classes)), name='weight')
b = tf.Variable(tf.random.normal((nb_classes,)), name='bias')
variables = [W, b]

#tf.nn.softmax computes softmax activations
#softmax = exp(logits) / reduce_sum(exp(logits), dim)
def logit_fn(X):
  return tf.matmul(X, W) + b

def hypothesis(X):
  return tf.nn.softmax(logit_fn(X))

def cost_fn(X, Y):
  logits = logit_fn(X)
  cost_i = tf.keras.losses.categorical_crossentropy(y_true=Y, y_pred=logits, from_logits=True)
  cost = tf.reduce_mean(cost_i)
  return cost

def grad_fn(X,Y):
  with tf.GradientTape() as tape:
    loss = cost_fn(X, Y)
    grads = tape.gradient(loss, variables)
    return grads

def prediction(X, Y):
  pred = tf.argmax(hypothesis(X), 1)
  correct_prediction = tf.equal(pred, tf.argmax(Y, 1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

  return accuracy
                        

In [10]:
def fit(X, Y, epochs=1000, verbose=100):
  optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)

  for i in range(epochs):
    grads = grad_fn(X, Y)
    optimizer.apply_gradients(zip(grads, variables))
    if (i==0) | ((i+1) % verbose == 0):
      print("{:5} | {:10.4f} | {:10.4f}".format(i+1, cost_fn(X, Y).numpy(), prediction(X, Y).numpy()))

fit(x_data, Y_one_hot)

    1 |     5.6101 |     0.1683
  100 |     0.6019 |     0.8416
  200 |     0.3819 |     0.9109
  300 |     0.2877 |     0.9406
  400 |     0.2310 |     0.9505
  500 |     0.1925 |     0.9604
  600 |     0.1644 |     0.9604
  700 |     0.1431 |     0.9604
  800 |     0.1264 |     0.9604
  900 |     0.1131 |     0.9901
 1000 |     0.1022 |     0.9901
